**Import libraries**

In [ ]:
pip install tensorflow[and-cuda]

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import tensorflow_datasets as tfds

**Import dataset**

In [3]:
# Load MNIST dataset 

mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
# This function computes the number of examples to be used for validation and test. 
# We do this by taking the split of the training dataset and splitting it into two sets one for validation and one for test
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [5]:
def scale(image, label):
  """
   Scale the image to 0 - 255. This is useful for image segmentation where you don't know the color of the image.
   
   @param image - A tensor of shape [ height width channels ] representing an image.
   @param label - A tensor of shape [ height width ] representing the ground truth labels.
   
   @return A tuple of the scaled image and label tensors. The first dimension is the same as the input and the second dimension is the same as the label
  """
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

In [6]:
# Scale data to be used for training and validation.
scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

In [7]:
# Randomly shuffles the training and validation data to avoid overfitting the data. 
# This is useful for generating random data
BUFFER_SIZE = 10000

suffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = suffled_train_and_validation_data.take(num_validation_samples)

train_data= suffled_train_and_validation_data.skip(num_validation_samples)

In [8]:
# Generate batches of training validation and test data for node index generation. This is to avoid running out of memory
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

In [9]:
# The function returns validation inputs and targets. 
# This is a generator function that iterates over the data and returns the first item
validation_inputs, validation_targets = next(iter(validation_data))

In [22]:
# This is the code that is run when you want to set up the parameters. 

input_size = 784
output_size = 10
hidden_layer_size = 200

In [23]:
# Build a Sequential model that can be used for training.
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(output_size, activation = 'softmax')
])

In [30]:
# Compiles and compiles Adamagnet model. Args : optimizer : Keras optimizer. learning_rate : Learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [31]:
# Train and validation model with training data and validation data. 
model.fit(train_data, epochs=5, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 10s - loss: 0.0256 - accuracy: 0.9920 - val_loss: 0.0325 - val_accuracy: 0.9895 - 10s/epoch - 19ms/step
Epoch 2/5
540/540 - 6s - loss: 0.0120 - accuracy: 0.9964 - val_loss: 0.0244 - val_accuracy: 0.9932 - 6s/epoch - 12ms/step
Epoch 3/5
540/540 - 6s - loss: 0.0080 - accuracy: 0.9978 - val_loss: 0.0170 - val_accuracy: 0.9955 - 6s/epoch - 12ms/step
Epoch 4/5
540/540 - 8s - loss: 0.0057 - accuracy: 0.9984 - val_loss: 0.0125 - val_accuracy: 0.9963 - 8s/epoch - 14ms/step
Epoch 5/5
540/540 - 7s - loss: 0.0043 - accuracy: 0.9987 - val_loss: 0.0085 - val_accuracy: 0.9968 - 7s/epoch - 12ms/step


In [32]:
# Evaluate the model on test_data and return the loss and accuracy.
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 2s 2s/step - loss: 0.1069 - accuracy: 0.9836


In [33]:
# Prints test loss and accuracy to standard output.
print('Test loss: {0:.2f}. Test Accuracy: {1:.2f}%'.format(test_loss, test_accuracy * 100.))

Test loss: 0.11. Test Accuracy: 98.36%
